In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import asyncio
from claude_agent_sdk import query, ClaudeAgentOptions

In [3]:
async def main():
    options = ClaudeAgentOptions(
        system_prompt="You are an expert Python developer",
        permission_mode='acceptEdits',
        cwd="/home/ahggg/desktop/github/side-project/aiswitch"
    )

    async for message in query(
        prompt="1+1=?",
        options=options
    ):
        print(message)


asyncio.run(main())

SystemMessage(subtype='init', data={'type': 'system', 'subtype': 'init', 'cwd': '/home/ahggg/desktop/github/side-project/aiswitch', 'session_id': '98e19697-609e-48e0-a9b8-2bf93daeca02', 'tools': ['Task', 'Bash', 'Glob', 'Grep', 'ExitPlanMode', 'Read', 'Edit', 'Write', 'NotebookEdit', 'WebFetch', 'TodoWrite', 'WebSearch', 'BashOutput', 'KillShell', 'SlashCommand'], 'mcp_servers': [], 'model': 'deepseek-chat', 'permissionMode': 'acceptEdits', 'slash_commands': ['compact', 'context', 'cost', 'init', 'output-style:new', 'pr-comments', 'release-notes', 'todos', 'review', 'security-review'], 'apiKeySource': 'none', 'output_style': 'default', 'agents': ['general-purpose', 'statusline-setup', 'output-style-setup'], 'uuid': 'e0b6c162-c575-48bc-9010-ca2cd3331254'})
AssistantMessage(content=[TextBlock(text='1+1=2')], model='deepseek-chat', parent_tool_use_id=None)
ResultMessage(subtype='success', duration_ms=6869, duration_api_ms=10791, is_error=False, num_turns=1, session_id='98e19697-609e-48e0-